# Решение задачи

Представленная задача решает ***следующую задачу:*** модель получает в качестве входных данных текст или текстовый файл, на основе которого происходит предсказание и выдача результата в виде процента схожести по стилистике написания текста в сравнении с  4 русскими авторами.

Установим Gradio, который будет учавствовать в создании интерфейса. Выполним установку токенизатора Razdel, загрузим предобученные Embedding. И установим большую базу текстов на русском языке - navec:

In [ ]:
!pip install --upgrade gradio
!pip install razdel
!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar
!pip install navec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.5/324.5 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 76.9 MB/s eta 0:00:00
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 1.10.1
    Uninstalling gradio_client-1.10.1:
      Successfully uninstalled gradio_client-1.10.1
  Attempting uninstall: gradio
    Found existing installation: gradio 5.31.0
    Uninstalling gradio-5.31.0:
      Successfully uninstalled gradio-5.31.0
--2025-07-18 13:49:51--  https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53012480 (51M) [application/x-tar]
Saving to: ‘navec_hudlit_v

Импортируем необходимые библиотеки и предопределим некоторые переменные:

In [ ]:
from sklearn.model_selection import train_test_split
from navec import Navec
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.models import Sequential, load_model
from keras.layers import Embedding, Dense, SpatialDropout1D, BatchNormalization, Dropout, Bidirectional, LSTM, GRU
import keras
from razdel import tokenize
import glob
import random
import numpy as np
from collections import defaultdict
import gradio as gr
import os

WIN_SIZE = 1000 #размер окна
WIN_STEP = 100 #шаг

Воспользуемся новым методом для нас загрузки датасета, описанным в официальной документации **Keras**. Данный метод позволяет избежать повторной загрузки, если данные были раньше скачаны. Метод возвращает путь к папке с датасетом.

Разархивируем датасета во временную папку *dataset:*

Отберем для обучения 12 писателей:

In [ ]:
data_path = keras.utils.get_file(
    "russian_literature.zip",
    "https://storage.yandexcloud.net/academy.ai/russian_literature.zip"
)

!unzip -qo "{data_path}" -d ./dataset

CLASS_LIST=["Dostoevsky", "Tolstoy", "Turgenev", "Chekhov", "Lermontov", "Blok", "Pushkin", "Gogol", "Gorky", "Herzen", "Bryusov", "Nekrasov" ]

21547079/21547079 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
error:  cannot create ./dataset/poems/Blok/╨б╤В╨╕╤Е╨╛╤В╨▓╨╛╤А╨╡╨╜╨╕╤П 1897-1903 ╨│╨│, ╨╜╨╡ ╨▓╨╛╤И╨╡╨┤╤И╨╕╨╡ ╨▓ ╨╛╤Б╨╜╨╛╨▓╨╜╨╛╨╡ ╤Б╨╛╨▒╤А╨░╨╜╨╕╨╡.txt
        File name too long
error:  cannot create ./dataset/prose/Gogol/╨Я╨╛╨▓╨╡╤Б╤В╤М ╨╛ ╤В╨╛╨╝, ╨║╨░╨║ ╨┐╨╛╤Б╤Б╨╛╤А╨╕╨╗╤Б╤П ╨Ш╨▓╨░╨╜ ╨Ш╨▓╨░╨╜╨╛╨▓╨╕╤З ╤Б ╨Ш╨▓╨░╨╜╨╛╨╝ ╨Э╨╕╨║╨╕╤Д╨╛╤А╨╛╨▓╨╕╤З╨╡╨╝.txt
        File name too long
error:  cannot create ./dataset/publicism/Tolstoy/╨Ф╨╛╨║╨╗╨░╨┤, ╨┐╤А╨╕╨│╨╛╤В╨╛╨▓╨╗╨╡╨╜╨╜╤Л╨╣ ╨┤╨╗╤П ╨║╨╛╨╜╨│╤А╨╡╤Б╤Б╨░ ╨╛ ╨╝╨╕╤А╨╡ ╨▓ ╨б╤В╨╛╨║╨│╨╛╨╗╤М╨╝╨╡.txt
        File name too long
error:  cannot create ./dataset/publicism/Tolstoy/╨Я╨╛╤З╨╡╨╝╤Г ╤Е╤А╨╕╤Б╤В╨╕╨░╨╜╤Б╨║╨╕╨╡ ╨╜╨░╤А╨╛╨┤╤Л ╨▓╨╛╨╛╨▒╤Й╨╡ ╨╕ ╨▓ ╨╛╤Б╨╛╨▒╨╡╨╜╨╜╨╛╤Б╤В╨╕ ╤А╤Г╤Б╤Б╨║╨╕╨╣ ╨╜╨░╤Е╨╛╨┤╤П╤В╤Б╤П ╤В╨╡╨┐╨╡╤А╤М ╨▓ ╨▒╨╡╨┤╤Б╤В╨▓╨╡╨╜╨╜╨╛╨╝ ╨┐╨╛╨╗╨╛╨╢╨╡╨╜╨╕╨╕.txt
        File name too long


Следующий код собирает текстовые данные из файлов в структуру словаря *all_texts*, где *ключи* — это имена авторов, а *значения* — объединённые тексты всех их произведений.

In [ ]:
all_texts = {} #собираем в словарь весь датасет

for author in CLASS_LIST:
    all_texts[author] = '' #инициализируем пустой строкой новый ключ словаря
    for path in glob.glob('./dataset/prose/{}/*.txt'.format(author)) +  glob.glob('./dataset/poems/{}/*.txt'.format(author)): #поиск файлов по шаблону
        with open(f'{path}', 'r', errors='ignore') as f: #игнорируем ошибки (например символы из другой кодировки)
            text = f.read() #загрузка содержимого файла в строку

        all_texts[author]  += ' ' + text.replace('\n', ' ') #заменяем символ перехода на новую строку пробелом

Далее выполним подсчёт популярности слов (токенов) в текстах из словаря *all_texts* и создадим словарь наиболее частых токенов (*word_index*), который можно использовать для векторизации текста.

**Razdel** — это библиотека для токенизации русского текста (разбиения на слова и знаки препинания).

Функция `razdel_tokenize(text)` принимает текст и возвращает список токенов (отдельных слов и символов).

In [ ]:
#реализация токенизатора Razdel
def razdel_tokenize(text):
    return [token.text for token in tokenize(text)]

token_counts = defaultdict(int)
max_words = 15000 #укажем размер словаря

#подсчет частотности токенов
for text in all_texts.values():
    tokens = razdel_tokenize(text.lower())
    for token in tokens:
        token_counts[token] += 1

sorted_tokens = sorted(token_counts.items(), key=lambda x: x[1], reverse=True)[:max_words ]#сортируем токены по частоте и берем top-N
word_index = {word: i+1 for i, (word, count) in enumerate(sorted_tokens)} #+1 потому что 0 зарезервирован

Преобразуем тексты из словаря *all_texts* в числовые последовательности, используя ранее созданный словарь *word_index*. Каждое слово заменяется на соответствующий ему числовой индекс, а неизвестные слова (которых нет в *word_index*) заменяются на 0:

In [ ]:
seq_train = []
for text in all_texts.values():
    tokens = razdel_tokenize(text)
    seq = [word_index.get(token, 0) for token in tokens if word_index.get(token, 0) < max_words]
    seq_train.append(seq)

total = sum(len(i) for i in seq_train) #вычисляем общее количество слов в датасете:

#общая выборка по писателям, их доля в базе данных, среднее и медианное значение слов
mean_list = np.array([])
for author in CLASS_LIST:
    cls = CLASS_LIST.index(author)
    mean_list = np.append(mean_list, len(seq_train[cls])) #собирает данные о длинах текстов для последующей балансировки датасета

#сбалансированная выборка
median = int(np.median(mean_list)) #зафиксировали медианное значение
seq_train_balance = []
CLASS_LIST_BALANCE = []
for author in CLASS_LIST:
    cls = CLASS_LIST.index(author)
    if len(seq_train[cls]) > median * 0.6:
        seq_train_balance.append(seq_train[cls][:median])
        CLASS_LIST_BALANCE.append(author)

#cлучайный выбор 4 писателей из сбалансированного списка
selected_authors = random.sample(CLASS_LIST_BALANCE, 4)
selected_seq_train_balance = [seq_train_balance[CLASS_LIST_BALANCE.index(author)] for author in selected_authors]

Далее реализуем 2 функции, а именно: **seq_split**, которая отвечает за разбиение последовательности чисел на небольшие "окна" фиксированной длины и **seq_vectorize**, которая осуществляет подготовку данных для нейросетей с разделением на обучение/валидацию/тест:

In [ ]:
def seq_split(sequence, win_size, step):
    return [sequence[i:i + win_size] for i in range(0, len(sequence) - win_size + 1, step)]

def seq_vectorize(seq_list, test_split, val_split, class_list, win_size, step):
    x_train, y_train = [], []
    x_val, y_val = [], []
    x_test, y_test = [], []

    #обрабатываем каждую последовательность
    for class_idx, seq in enumerate(seq_list):
        #вычисляем границы разделения последовательности
        total_len = len(seq)
        val_end = int(total_len * (1 - test_split - val_split))
        test_end = int(total_len * (1 - test_split))

        #разбиваем последовательность на окна для каждой части
        train_windows = seq_split(seq[:val_end], win_size, step)
        val_windows = seq_split(seq[val_end:test_end], win_size, step)
        test_windows = seq_split(seq[test_end:], win_size, step)

        #добавляем окна в соответствующие списки
        x_train += train_windows
        x_val += val_windows
        x_test += test_windows

        #создаем one-hot метки и добавляем их в соответствующие списки
        #количество меток равно количеству окон
        y_train += [keras.utils.to_categorical(class_idx, len(class_list))] * len(train_windows)
        y_val += [keras.utils.to_categorical(class_idx, len(class_list))] * len(val_windows)
        y_test += [keras.utils.to_categorical(class_idx, len(class_list))] * len(test_windows)

    #преобразуем списки в numpy массивы и возвращаем результат
    return (np.array(x_train), np.array(y_train),
            np.array(x_val), np.array(y_val),
            np.array(x_test), np.array(y_test))

Выполним финальную подготовку модели, создав функцию `build_and_train_model`.

Преобразуем тексты в числовой формат и разделяя их на обучающую, валидационную и тестовую выборки.

Загрузим предобученную модель, инициализируем матрицу эмбеддингов и заполним матрицы векторами слов:

`hudlit_12B_500K_300d_100q` — это **GloVe-эмбединги** обученные на 145ГБ художественной литературы. Архив с текстами взят из проекта **RUSSE** и использует оригинальную реализацию **GloVe** на C, обернутую в удобный Python-интерфейс.

Размер словаря `hudlit_12B_500K_300d_100q` — 500 000 записей, он покрывает 98% слов в художественных текстах. Оптимальная размерность векторов — 300. Таблица 500 000 × 300 из float-чисел занимает 578МБ, размер архива с весами `hudlit_12B_500K_300d_100q` в 12 раз меньше (48МБ). И это благодаря квантизации.

Выполним настройку архитектуры добавим callbacks, осуществим компиляцию модели, её обучение и дальнейшее сохранение:

In [ ]:
model = None #глобальная переменная для хранения модели

def build_and_train_model():
    global model
    x_train, y_train, x_val, y_val, x_test, y_test = seq_vectorize(
        selected_seq_train_balance,
        test_split=0.1,
        val_split=0.1,
        class_list=selected_authors,
        win_size=WIN_SIZE,
        step=WIN_STEP
    )
    navec = Navec.load('navec_hudlit_v1_12B_500K_300d_100q.tar')
    embedding_matrix = np.zeros((max_words, 300))
    for word, i in word_index.items():
        if i < max_words:
            embedding_vector = navec.get(word)
            if embedding_vector is not None: #слова, которых нет в Navec (или с индексом >= max_words), получают нулевой вектор
                embedding_matrix[i] = embedding_vector
    model_local = Sequential()
    model_local.add(Embedding(max_words, 300, input_length=WIN_SIZE, weights=[embedding_matrix]))
    model_local.add(SpatialDropout1D(0.3))
    model_local.add(BatchNormalization())
    model_local.add(Bidirectional(LSTM(32, return_sequences=True)))
    model_local.add(Bidirectional(LSTM(32, return_sequences=True)))
    model_local.add(Dropout(0.3))
    model_local.add(BatchNormalization())
    model_local.add(GRU(32, return_sequences=True, reset_after=True))
    model_local.add(GRU(32, reset_after=True))
    model_local.add(Dropout(0.3))
    model_local.add(BatchNormalization())
    model_local.add(Dense(100, activation='relu'))
    model_local.add(Dropout(0.3))
    model_local.add(BatchNormalization())
    model_local.add(Dense(len(selected_authors), activation='softmax'))
    x_train_final, x_val_final, y_train_final, y_val_final = train_test_split(
        x_train, y_train, test_size=0.1, random_state=42, stratify=y_train
    )
    early_stopping = EarlyStopping(
        monitor='val_accuracy',
        patience=3,
        restore_best_weights=True,
        verbose=0
    )
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.3,
        patience=3,
        verbose=0
    )
    checkpoint = ModelCheckpoint(
        'best_model.keras',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=0
    )
    optimizer = keras.optimizers.Adam(learning_rate=0.001)
    model_local.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model_local.fit(
        x_train_final, y_train_final,
        epochs=30,
        batch_size=128,
        validation_data=(x_val_final, y_val_final),
        callbacks=[early_stopping, reduce_lr, checkpoint],
        verbose=0
    )
    model_local = load_model('best_model.keras')
    model = model_local

Создадим функцию для интерфейса Gradio, которая запускает предыдущую функцию и сигнализирует об успешном завершении обучения:

In [ ]:
def train_model_gradio():
    build_and_train_model()
    return gr.update(value="Обучение завершено!", interactive=False), gr.update(interactive=True)

***Общее назначение функции:***

Функция принимает текст (или файл), обрабатывает его, преобразует в формат для нейросети, запускает предсказание и возвращает вероятности того, на какого из писателей больше всего похож представленный текст

***Основные этапы:***

1.   Проверка, обучена ли модель;
2.   Получение текста для анализа.;
3.   Проверка минимальной длины текста;
4.   Токенизация и преобразование в индексы;
5.   Формирование окон для подачи на модель;
6.   Получение и усреднение предсказаний;
7.   Формирование и возврат результата.









In [ ]:
def predict_author(text, file):
    global model  #используем глобальную переменную model, чтобы проверить, обучена ли модель и сделать предсказание
    if model is None:
        #если модель еще не обучена, возвращаем нулевые вероятности для всех авторов
        return {author: 0.0 for author in selected_authors}

    #получаем текст для анализа: приоритет — текст из поля, иначе из загруженного файла
    if text and text.strip() != "":
        content = text
    elif file is not None:
        #пытаемся прочитать содержимое файла
        try:
            with open(file.name, 'r', encoding='utf-8') as f:
                content = f.read()
        except Exception as e:
            #в случае ошибки при чтении файла возвращаем нулевые вероятности
            return {author: 0.0 for author in selected_authors}
    else:
        #если не был передан ни текст, ни файл — возвращаем нулевые вероятности
        return {author: 0.0 for author in selected_authors}

    #проверяем, что текст достаточно длинный для анализа (минимум 500 символов)
    if len(content) < 500:
        return {author: 0.0 for author in selected_authors}

    #токенизация текста (разделяем на слова)
    tokens = razdel_tokenize(content.lower())
    #преобразование токенов в числовые индексы для подачи на вход модели
    seq = [word_index.get(token, 0) for token in tokens]

    #формируем окна фиксированной длины для подачи на модель
    if len(seq) < WIN_SIZE:
        #если текст слишком короткий, дополняем последовательность нулями до нужной длины
        seq_padded = seq + [0] * (WIN_SIZE - len(seq))
        windows = [seq_padded]
    else:
        #если текст достаточно длинный, делим на окна с шагом WIN_STEP
        windows = []
        for i in range(0, len(seq) - WIN_SIZE + 1, WIN_STEP):
            windows.append(seq[i:i + WIN_SIZE])
            if len(windows) >= 10:  #ограничиваем количество окон для ускорения инференса
                break

    windows = np.array(windows)  #переводим список окон в numpy-массив

    #получаем предсказания модели для каждого окна
    predictions = model.predict(windows, verbose=0)
    #усредняем вероятности по всем окнам текста
    avg_probs = np.mean(predictions, axis=0)

    #формируем результат: словарь {автор: вероятность}
    result = {selected_authors[i]: float(avg_probs[i]) for i in range(len(selected_authors))}
    return result  #возвращаем вероятности для каждого автора

Далее генерируются примеры текстов для тестирования, создается описание для пользователя, добавляются в интерфейс кнопки для обучения и предсказания, а так же поля для ввода текста/файла и блок с примерами:

In [ ]:
#формируем список примеров для интерфейса Gradio
examples = []
for author in selected_authors:
    text_sample = all_texts[author]  #берем полный текст автора
    #случайно выбираем фрагмент длиной 1000 символов для примера
    start_idx = random.randint(0, len(text_sample) - 1000)
    examples.append([text_sample[start_idx:start_idx + 1000], None])

#описание интерфейса и инструкции для пользователя
description = f"""
## Классификатор авторства литературных текстов
Модель определяет авторство среди {len(selected_authors)} русских писателей:
{', '.join(selected_authors)}

**Инструкция:**
1. Нажмите "Обучить модель", дождитесь завершения обучения;
2. Введите текст в поле ниже (минимум 500 символов) или загрузите файл;
3. Нажмите кнопку "Выполнить предсказание";
4. Результат покажет вероятности для каждого автора.

**Советы:**
- Используйте примеры ниже для тестирования;
- Чем длиннее текст (1-2 тыс. символов), тем точнее результат.
"""

#создаем интерфейс Gradio в блоке
with gr.Blocks(theme=gr.themes.Soft()) as iface:
    gr.Markdown("# Классификатор литературных текстов")
    gr.Markdown(description)
    #кнопка обучения и статус - первая строка интерфейса
    with gr.Row():
        btn_train = gr.Button("Обучить модель")
        train_status = gr.Textbox(label="Статус обучения", value="Модель не обучена", interactive=False)
    #ввод текста или загрузка файла - вторая строка интерфейса
    with gr.Row():
        input_text = gr.Textbox(
            lines=8,
            placeholder="Введите текст здесь...",
            label="Текст для анализа"
        )
        input_file = gr.File(label="Или загрузите текстовый файл (.txt)")
    #блок вывода результата
    output_label = gr.Label(
        label="Результаты распознавания",
        num_top_classes=3
    )
    #кнопка для запуска предсказания (неактивна до обучения)
    btn_predict = gr.Button("Выполнить предсказание", interactive=False)
    #примеры для быстрого теста интерфейса
    gr.Examples(
        examples=examples,
        inputs=[input_text, input_file],
        outputs=output_label,
        label="Примеры",
        examples_per_page=4
    )
    #обработка нажатия кнопки обучения: запускает обучение модели
    btn_train.click(
        fn=train_model_gradio,
        inputs=[],
        outputs=[train_status, btn_predict]
    )
    #обработка нажатия кнопки предсказания: запускает инференс
    btn_predict.click(
        fn=predict_author,
        inputs=[input_text, input_file],
        outputs=output_label
    )

#запуск интерфейса Gradio
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://29a75c57e8f45ffc1d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
